In [62]:
%load_ext autoreload
%autoreload 2
import os, sys
os.environ['STREAM_LOG_LEVEL'] = 'ERROR'
os.environ['FILE_LOG_LEVEL'] = 'DEBUG'
os.environ['PROPAGATE_TO_ROOT_LOGGER'] = 'False'
os.environ['PROPAGATE_TO_ROOT_LOGGER'], os.environ['STREAM_LOG_LEVEL']
from trade.assets.Stock import Stock
from trade.assets.Option import Option
from trade.assets.OptionStructure import OptionStructure
from trade.helpers.Context import Context, clear_context
from trade.helpers.helper import (change_to_last_busday, 
                                  is_USholiday, 
                                  is_busday, 
                                  setup_logger, 
                                  generate_option_tick, 
                                  get_option_specifics_from_key,
                                  identify_length,
                                  extract_numeric_value)
from scipy.stats import percentileofscore
from EventDriven.riskmanager import RiskManager
from dbase.DataAPI.ThetaData import (list_contracts, retrieve_openInterest, retrieve_eod_ohlc, retrieve_quote)
from pandas.tseries.offsets import BDay
from itertools import product
import pandas as pd
from copy import deepcopy
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathos.multiprocessing import ProcessingPool as Pool
import numpy as np
import time
chain_cache = {}
close_cache = {}
spot_cache = {}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
list_contracts('AAPL', '2024-03-12')

,root,expiration,strike,right
0,AAPL,20240621,225.0,C
1,AAPL,20240719,225.0,C
2,AAPL,20250620,220.0,C
3,AAPL,20240816,225.0,C
4,AAPL,20240621,230.0,C
...,...,...,...,...
1040,AAPL,20250117,215.0,C
1041,AAPL,20250117,215.0,P
1042,AAPL,20251219,210.0,C
1043,AAPL,20240412,220.0,C


In [63]:
rm = RiskManager(None, None, 1000000)
tick = 'TSLA'
date = '2023-07-24'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'
order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.15},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.15} 
            ],
            'name': 'vertical_spread'
        }
er = rm.OrderPicker.get_order(tick, date, 'C', 10, order_settings)
er

In [61]:
def produce_order_candidates(settings, tick, date, right = 'P'):
    order_candidates = {'long': [], 'short': []}
    for spec in settings['specifics']:
        order_candidates[spec['direction']].append(chain_details(date, tick, spec['dte'], spec['rel_strike'], right,  moneyness_width = spec['moneyness_width']))
    return order_candidates


def liquidity_check(id, date, pass_threshold = 250):
    sample_id = deepcopy(get_option_specifics_from_key(id))
    new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
    transfer_dict = {}
    for k, v in sample_id.items():

        if k in new_dict_keys:
            if k == 'strike':
                transfer_dict[new_dict_keys[k]] = float(sample_id[k])
            else:
                transfer_dict[new_dict_keys[k]] = sample_id[k]

    start = (pd.to_datetime(date) - BDay(10)).strftime('%Y-%m-%d')
    oi_data = retrieve_openInterest(**transfer_dict, end_date=date, start_date=start)
    # print(f'Open Interest > {pass_threshold} for {id}:', oi_data.Open_interest.mean() )
    return oi_data.Open_interest.mean() > pass_threshold


def available_close_check(id, date, threshold = 0.7):
    cache_key = f"{id}_{date}"
    sample_id = deepcopy(get_option_specifics_from_key(id))
    new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
    transfer_dict = {}
    for k, v in sample_id.items():
        if k in new_dict_keys:
            if k == 'strike':
                transfer_dict[new_dict_keys[k]] = float(sample_id[k])
            else:
                transfer_dict[new_dict_keys[k]] = sample_id[k]
    
    if cache_key in close_cache:
        close_data_sample = close_cache[cache_key]
    else:
        start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
        close_data_sample = retrieve_eod_ohlc(**transfer_dict, start_date=start, end_date=date)
        close_cache[cache_key] = close_data_sample
    close_mask_series = close_data_sample.Close != 0
    return close_mask_series.sum()/len(close_mask_series) > threshold


def get_structure_price(tradeables, direction_index, date, tick, right = 'P'):
    pack_price = {}
    pack_dataframe = pd.DataFrame()
    pack_dataframe['close'] = 0

    for pack_i, pack in enumerate(tradeables):
        pack_close = 0
        for i, id in enumerate(pack):
            if id not in spot_cache:
                
                cache_key = f"{id}_{date}"
                sample_id = deepcopy(get_option_specifics_from_key(id))
                new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
                transfer_dict = {}
                for k, v in sample_id.items():
                    if k in new_dict_keys:
                        if k == 'strike':
                            transfer_dict[new_dict_keys[k]] = float(sample_id[k])
                        else:
                            transfer_dict[new_dict_keys[k]] = sample_id[k]
                start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
                close_data_sample = retrieve_eod_ohlc(**transfer_dict, start_date=start, end_date=date)
                close_data_sample = close_data_sample[~close_data_sample.index.duplicated(keep = 'first')]
                close = close_data_sample['Midpoint'][date]
                spot_cache[cache_key] = close
            else:
                close = cache_key[id]
            pack_close += close * direction_index[i]
            pack_dataframe.at[pack_i, i] = id

        pack_dataframe.at[pack_i, 'close'] = pack_close
    return pack_dataframe




In [8]:
def chain_details(date, ticker, tgt_dte, tgt_moneyness, right = 'P', moneyness_width = 0.15, print_stderr = False):
    return_dataframe = pd.DataFrame()
    errors = {}
    if not (is_USholiday(date) and not is_busday(date)):
        try:
            print(date, ticker) if print_stderr else None
            ## Get both calls and puts per moneyness. For 1 Moneyness, both will most be available. If not, if one is False, other True. 
            ## We will need to get two rows. 
            chain_key = f"{date}_{ticker}"
            with Context(end_date = date):
                if chain_key in chain_cache:
                    Option_Chain = chain_cache[chain_key]
                else:
                    start_time = time.time()
                    Stock_obj = Stock(ticker, run_chain = False)
                    end_time = time.time()
                    print(f"Time taken to get stock object: {end_time-start_time}") if print_stderr else None
                    Option_Chain = Stock_obj.option_chain()
                    Spot = Stock_obj.spot(ts = False)
                    Spot = list(Spot.values())[0]
                    Option_Chain['Spot'] = Spot
                    Option_Chain['q'] = Stock_obj.div_yield()
                    Option_Chain['r'] = Stock_obj.rf_rate
                    chain_cache[chain_key] = Option_Chain

                
                Option_Chain_Filtered = Option_Chain[Option_Chain[right.upper()] == True]
                
                
                if right == 'P':
                    Option_Chain_Filtered['relative_moneyness']  = Option_Chain_Filtered.index.get_level_values('strike')/Option_Chain_Filtered.Spot
                elif right == 'C':
                    Option_Chain_Filtered['relative_moneyness']  = Option_Chain_Filtered.Spot/Option_Chain_Filtered.index.get_level_values('strike')
                else:
                    raise ValueError(f'Right dne. recieved {right}')
                Option_Chain_Filtered['moneyness_spread'] = (tgt_moneyness-Option_Chain_Filtered['relative_moneyness'])**2
                Option_Chain_Filtered['dte_spread'] = (Option_Chain_Filtered.index.get_level_values('DTE')-tgt_dte)**2
                Option_Chain_Filtered.sort_values(by=['dte_spread','moneyness_spread'], inplace = True)
                Option_Chain_Filtered = Option_Chain_Filtered.loc[Option_Chain_Filtered['dte_spread'] == Option_Chain_Filtered['dte_spread'].min()]
                if float(moneyness_width) == 0.0:
                    option_details = Option_Chain_Filtered.sort_values('moneyness_spread', ascending=False).head(1)
                else:
                    option_details = Option_Chain_Filtered[(Option_Chain_Filtered['relative_moneyness'] >= tgt_moneyness-moneyness_width) & 
                                                        (Option_Chain_Filtered['relative_moneyness'] <= tgt_moneyness+moneyness_width)]
                option_details['build_date'] = date
                option_details['ticker'] = ticker
                option_details['moneyness'] = tgt_moneyness
                option_details['TGT_DTE'] = tgt_dte
                option_details.reset_index(inplace = True)
                option_details.set_index('build_date', inplace = True)
                option_details['right'] = right
                option_details.drop(columns = ['C','P'], inplace = True)
                option_details['option_id'] = option_details.apply(lambda x: generate_option_tick(symbol = x['ticker'], 
                                                                    exp = x['expiration'].strftime('%Y-%m-%d'), strike = float(x['strike']), right = x['right']), axis = 1)
                return_dataframe = pd.concat([return_dataframe, option_details])
            clear_context()
            return_dataframe.drop_duplicates(inplace = True)

        except Exception as e:
            raise
            errors[date] = e
            return errors
        return return_dataframe.sort_values('relative_moneyness', ascending=False)
    else:
        return None, errors
    

details= chain_details('2024-03-12', 'TSLA', 365, 0.7, moneyness_width = 0.00)

In [9]:
class OrderPicker:
    def __init__(self):
        self.liquidity_threshold = 250
        self.data_availability_threshold = 0.7
        self.lookback = 30

    def get_order(self, 
                  tick, 
                  date,
                  right, 
                  max_close,
                  order_settings):
        
        ## Create necessary data structures
        direction_index = {}
        for indx, v in enumerate(order_settings['specifics']):
            if v['direction'] == 'long':
                direction_index[indx] = 1
            elif v['direction'] == 'short':
                direction_index[indx] = -1

        ## Produce Order Candidates
        start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
        order_candidates = produce_order_candidates(order_settings, tick, date, right)

        ## Check Liquidity and Close Availability, Filter out those that don't meet the criteria
        for direction in order_candidates:
            for i,data in enumerate(order_candidates[direction]):
                data['liquidity_check'] = data.option_id.apply(lambda x: liquidity_check(x, date))
                order_candidates[direction][i] = data[data.liquidity_check == True]

        for direction in order_candidates:
            for i,data in enumerate(order_candidates[direction]):
                data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))
                order_candidates[direction][i] = data[data.available_close_check == True] 

        ## Filter Unique Combinations per leg.
        unique_ids = {'long': [], 'short': []}
        for direction in order_candidates:
            for i,data in enumerate(order_candidates[direction]):
                unique_ids[direction].append(data[(data.liquidity_check == True) & (data.available_close_check == True)].option_id.unique().tolist())
        
        ## Produce Tradeable Combinations
        tradeable_ids = list(product(*unique_ids['long'], *unique_ids['short']))
        tradeable_ids, unique_ids 
        
        ## Keep only unique combinations. Not repeating a contract.
        filtered = [t for t in tradeable_ids if len(set(t)) == len(t)]

        ## Get the price of the structure
        prices = get_structure_price(filtered, direction_index, date, 'AAPL')

        ## Return the structure with the best price
        return_dataframe = prices[(prices.close<= max_close)].sort_values('close', ascending = False).head(1)
        return_order = {'long': [], 'short': []}
        id = ''
        for key, v in direction_index.items():
            if v < 0:
                option_id = return_dataframe[key].values[0]
                id += f'&L:{option_id}'
                return_order['short'].append(option_id)
            elif v > 0:
                option_id = return_dataframe[key].values[0]
                id += f'&S:{option_id}'
                return_order['long'].append(option_id)
        
        return_order['trade_id'] = id
        return return_order


order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.15},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.15} 
            ],
            'name': 'vertical_spread'
        }


tick = 'TSLA'
date = '2024-03-12'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'P'


picker = OrderPicker()
er = picker.get_order(tick, date, 'C', 10, order_settings)

In [10]:
class RiskManager:
    def __init__(self,
                 bars,
                 events,
                 initial_capital,
                 ):
        self.bars = bars
        self.events = events
        self.initial_capital = initial_capital
        # self.symbol_list = self.bars.symbol_list
        self.OrderSelector = OrderPicker()


    def get_order(self, symbol, date, order_settings):
        pass



# rm = RiskManager(None, None, 1000000)
# rm.OrderSelector.get_order(tick, date, 'C', 10, order_settings)

In [41]:
chain_cache.keys()

dict_keys(['2023-07-24_TSLA'])

In [12]:
details

right,expiration,DTE,strike,Spot,q,r,relative_moneyness,moneyness_spread,dte_spread,ticker,moneyness,TGT_DTE,right,option_id
build_date,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,330.0,177.539993,0,0.0524,1.858736,1.342669,64,TSLA,0.7,365,P,TSLA20250321000330P


In [25]:
import pandas as pd
from pandas.tseries.offsets import BDay
import asyncio
from aiohttp import ClientSession

# Mock asynchronous functions for openInterest and OHLC retrieval
async def retrieve_openInterest_async(session, **kwargs):
    return retrieve_openInterest(**kwargs)

async def retrieve_eod_ohlc_async(session, **kwargs):
    # Replace this with actual asynchronous logic (e.g., an API call)
    return retrieve_eod_ohlc(**kwargs)

async def fetch_all(records):
    async with ClientSession() as session:
        # Gather all OpenInterest tasks
        open_interest_tasks = [
            retrieve_openInterest_async(session, **record) for record in records
        ]
        # Gather all EOD OHLC tasks
        eod_ohlc_tasks = [
            retrieve_eod_ohlc_async(session, **record) for record in records
        ]
        
        # Execute all tasks concurrently
        open_interest_results = await asyncio.gather(*open_interest_tasks)
        eod_ohlc_results = await asyncio.gather(*eod_ohlc_tasks)
        
        return open_interest_results, eod_ohlc_results

import nest_asyncio
nest_asyncio.apply()

def populate_cache(order_candidates, date='2024-03-12'):
    full_data = pd.DataFrame()
    for direction in order_candidates:
        for i, data in enumerate(order_candidates[direction]):
            full_data = pd.concat([full_data, data])
    
    start = (pd.to_datetime(date) - BDay(20)).strftime('%Y-%m-%d')
    full_data.drop_duplicates(inplace=True) 
    full_data[['right', 'exp', 'strike', 'symbol']] = full_data[['right', 'expiration', 'strike', 'ticker']]
    full_data[['end_date', 'start_date']] = date, start
    records = full_data[['right', 'exp', 'strike', 'symbol', 'end_date', 'start_date']].to_dict(orient='records')

    # Run the asyncio event loop
    loop = asyncio.get_event_loop()
    futures, futures2 = loop.run_until_complete(fetch_all(records))

    return futures, futures2



# populate_cache(order_candidates, date='2024-03-12')

In [16]:
def populate_cache(order_candidates, date = '2024-03-12'):
    full_data = pd.DataFrame()
    for direction in order_candidates:
        for i,data in enumerate(order_candidates[direction]):
            full_data = pd.concat([full_data, data])
    
    start = (pd.to_datetime(date) - BDay(20)).strftime('%Y-%m-%d')
    full_data.drop_duplicates(inplace = True) 
    full_data[['right', 'exp', 'strike', 'symbol']] = full_data[['right', 'expiration', 'strike', 'ticker']]
    full_data[['end_date', 'start_date']] = date, start
    records = full_data[['right', 'exp', 'strike', 'symbol', 'end_date', 'start_date']].to_dict(orient = 'records')

    with ThreadPoolExecutor(max_workers = 10) as executor:
        futures = [executor.submit(retrieve_openInterest, **records[i]).result() for i in range(len(records))]
        futures2 = [executor.submit(retrieve_eod_ohlc, **records[i]).result() for i in range(len(records))]




    return futures, futures2
to_populate_sample = populate_cache(order_candidates)

In [17]:
# to_populate_sample[1]

In [34]:
to_populate_sample[['right', 'expiration', 'strike', 'ticker']].to_dict(orient = 'records')
to_populate_sample[['right', 'exp', 'strike', 'symbol']] = to_populate_sample[['right', 'expiration', 'strike', 'ticker']]
to_populate_sample[['end_date', 'start_date']] = date
to_populate_sample[['right', 'exp', 'strike', 'symbol', 'end_date', 'start_date']].to_dict(orient = 'records')

[{'right': 'P',
  'exp': Timestamp('2025-03-21 00:00:00'),
  'strike': 190.0,
  'symbol': 'AAPL',
  'end_date': '2024-03-12',
  'start_date': '2024-03-12'},
 {'right': 'P',
  'exp': Timestamp('2025-03-21 00:00:00'),
  'strike': 185.0,
  'symbol': 'AAPL',
  'end_date': '2024-03-12',
  'start_date': '2024-03-12'},
 {'right': 'P',
  'exp': Timestamp('2025-03-21 00:00:00'),
  'strike': 180.0,
  'symbol': 'AAPL',
  'end_date': '2024-03-12',
  'start_date': '2024-03-12'},
 {'right': 'P',
  'exp': Timestamp('2025-03-21 00:00:00'),
  'strike': 175.0,
  'symbol': 'AAPL',
  'end_date': '2024-03-12',
  'start_date': '2024-03-12'},
 {'right': 'P',
  'exp': Timestamp('2025-03-21 00:00:00'),
  'strike': 170.0,
  'symbol': 'AAPL',
  'end_date': '2024-03-12',
  'start_date': '2024-03-12'},
 {'right': 'P',
  'exp': Timestamp('2025-03-21 00:00:00'),
  'strike': 165.0,
  'symbol': 'AAPL',
  'end_date': '2024-03-12',
  'start_date': '2024-03-12'},
 {'right': 'P',
  'exp': Timestamp('2025-03-21 00:00:00'),

In [29]:
to_populate_sample.columns

Index(['expiration', 'DTE', 'strike', 'Spot', 'q', 'r', 'relative_moneyness',
       'moneyness_spread', 'dte_spread', 'ticker', 'moneyness', 'TGT_DTE',
       'right', 'option_id', 'liquidity_check', 'available_close_check'],
      dtype='object', name='right')

In [13]:
order_candidates['long'][0]

right,expiration,DTE,strike,Spot,q,r,relative_moneyness,moneyness_spread,dte_spread,ticker,moneyness,TGT_DTE,right,option_id
build_date,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,190.0,173.229996,0.005542,0.0524,1.096808,0.009372,64,AAPL,1.0,365,P,AAPL20250321000190P
2024-03-12,2025-03-21,373,185.0,173.229996,0.005542,0.0524,1.067944,0.004616,64,AAPL,1.0,365,P,AAPL20250321000185P
2024-03-12,2025-03-21,373,180.0,173.229996,0.005542,0.0524,1.039081,0.001527,64,AAPL,1.0,365,P,AAPL20250321000180P
2024-03-12,2025-03-21,373,175.0,173.229996,0.005542,0.0524,1.010218,0.000104,64,AAPL,1.0,365,P,AAPL20250321000175P
2024-03-12,2025-03-21,373,170.0,173.229996,0.005542,0.0524,0.981354,0.000348,64,AAPL,1.0,365,P,AAPL20250321000170P
2024-03-12,2025-03-21,373,165.0,173.229996,0.005542,0.0524,0.952491,0.002257,64,AAPL,1.0,365,P,AAPL20250321000165P
2024-03-12,2025-03-21,373,160.0,173.229996,0.005542,0.0524,0.923628,0.005833,64,AAPL,1.0,365,P,AAPL20250321000160P


In [ ]:
order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.1},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.1} 
            ],
            'name': 'vertical_spread'
        }


tick = 'TSLA'
date = '2023-07-24'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'

direction_index = {}
for indx, v in enumerate(order_settings['specifics']):
    if v['direction'] == 'long':
        direction_index[indx] = 1
    elif v['direction'] == 'short':
        direction_index[indx] = -1

direction_index


{0: 1, 1: -1}

In [ ]:
order_candidates = produce_order_candidates(order_settings, tick, date, right)

In [42]:
# for direction in order_candidates:
#     for i,data in enumerate(order_candidates[direction]):
#         data['liquidity_check'] = data.option_id.apply(lambda x: liquidity_check(x, date))
#         data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))
#         order_candidates[direction][i] = data[data.liquidity_check == True]

# for direction in order_candidates:
#     for i,data in enumerate(order_candidates[direction]):
#         data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))
#         order_candidates[direction][i] = data[data.available_close_check == True]  


for direction in order_candidates:
    for i,data in enumerate(order_candidates[direction]):
        data['liquidity_check'] = data.option_id.apply(lambda x: liquidity_check(x, date))
        data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))


## Filter Unique Combinations per leg.

unique_ids = {'long': [], 'short': []}
for direction in order_candidates:
    for i,data in enumerate(order_candidates[direction]):
        unique_ids[direction].append(data.option_id.unique())

tradeable_ids = list(product(*unique_ids['long'], *unique_ids['short']))
  

In [44]:
filtered = [t for t in tradeable_ids if len(set(t)) == len(t)]
pd.DataFrame(filtered)
filtered

[('TSLA20240621024667C', 'TSLA20240621028333C'),
 ('TSLA20240621024667C', 'TSLA20240621028667C'),
 ('TSLA20240621024667C', 'TSLA20240621000290C'),
 ('TSLA20240621024667C', 'TSLA20240621029333C'),
 ('TSLA20240621024667C', 'TSLA20240621029667C'),
 ('TSLA20240621024667C', 'TSLA20240621000300C'),
 ('TSLA20240621024667C', 'TSLA20240621030333C'),
 ('TSLA20240621024667C', 'TSLA20240621030667C'),
 ('TSLA20240621024667C', 'TSLA20240621000310C'),
 ('TSLA20240621024667C', 'TSLA20240621031667C'),
 ('TSLA20240621024667C', 'TSLA20240621000320C'),
 ('TSLA20240621024667C', 'TSLA20240621032667C'),
 ('TSLA20240621024667C', 'TSLA20240621000330C'),
 ('TSLA20240621024667C', 'TSLA20240621033333C'),
 ('TSLA20240621024667C', 'TSLA20240621033667C'),
 ('TSLA20240621024667C', 'TSLA20240621000340C'),
 ('TSLA20240621024667C', 'TSLA20240621034333C'),
 ('TSLA20240621024667C', 'TSLA20240621034667C'),
 ('TSLA20240621024667C', 'TSLA20240621000350C'),
 ('TSLA20240621024667C', 'TSLA20240621035333C'),
 ('TSLA2024062102466

In [ ]:
prices = get_structure_price(filtered, direction_index, date, 'AAPL')
prices


In [45]:
return_dataframe = prices[(prices.close<= 5) & (prices.close>0)].sort_values('close', ascending = False).head(1)
return_order = {'long': [], 'short': []}
for key, v in direction_index.items():
    if v < 0:
        return_order['short'].append(return_dataframe[key].values[0])
    elif v > 0:
        return_order['long'].append(return_dataframe[key].values[0])
return_order

{'long': ['AAPL20250620000230C'], 'short': ['AAPL20250620000240C']}

In [37]:
prices#[prices.close<= 5]
prices#[(prices.close<= 5) & (prices.close>0)].sort_values('close', ascending = False)

,0,1,close
0,AAPL20250321000180P,AAPL20250321000160P,7.95
1,AAPL20250321000180P,AAPL20250321000150P,10.55
2,AAPL20250321000180P,AAPL20250321000140P,12.40
3,AAPL20250321000175P,AAPL20250321000160P,5.55
4,AAPL20250321000175P,AAPL20250321000150P,8.15
5,AAPL20250321000175P,AAPL20250321000140P,10.00
6,AAPL20250321000170P,AAPL20250321000160P,3.45
7,AAPL20250321000170P,AAPL20250321000150P,6.05
8,AAPL20250321000170P,AAPL20250321000140P,7.90
9,AAPL20250321000165P,AAPL20250321000160P,1.60


In [20]:
prices[prices.close<= 5]

,0,1,close
6,AAPL20250321000170P,AAPL20250321000160P,3.45
9,AAPL20250321000165P,AAPL20250321000160P,1.60
10,AAPL20250321000165P,AAPL20250321000150P,4.20
12,AAPL20250321000160P,AAPL20250321000150P,2.60
13,AAPL20250321000160P,AAPL20250321000140P,4.45


In [21]:
min(prices.values())

TypeError: 'numpy.ndarray' object is not callable

In [275]:
order_candidates['long'][0]

right,expiration,DTE,strike,relative_moneyness,moneyness_spread,dte_spread,ticker,Spot,moneyness,TGT_DTE,q,r,right,option_id,liquidity_check,available_close_check
build_date,,,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,180.0,1.039081,0.001527,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000180P,True,True
2024-03-12,2025-03-21,373,170.0,0.981354,0.000348,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000170P,True,True
2024-03-12,2025-03-21,373,165.0,0.952491,0.002257,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000165P,True,True


In [277]:
order_candidates['short'][0]

right,expiration,DTE,strike,relative_moneyness,moneyness_spread,dte_spread,ticker,Spot,moneyness,TGT_DTE,q,r,right,option_id,liquidity_check,available_close_check
build_date,,,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,150.0,0.865901,0.000253,64,AAPL,173.229996,0.85,365,0.005542,0.0524,P,AAPL20250321000150P,True,True


In [260]:
structure = {
    'long': [{'strike': 165.0, 'expiration': '2025-03-21', 'underlier': 'AAPL', 'right': 'p' }],
    'short': [{'strike': 150.0, 'expiration': '2025-03-21', 'underlier': 'AAPL', 'right': 'p' }]
}

with Context(end_date = date):
    struct_obj = OptionStructure(structure)

In [262]:
struct_obj.pv

4.199999999999999

In [227]:
transfer_dict

{'symbol': 'AAPL', 'right': 'P', 'exp': '2025-03-21', 'strike': 165.0}

In [74]:
retrieve_quote(**transfer_dict, end_date = date, start_date=date, start_time = '9:30',)

NameError: name 'transfer_dict' is not defined

In [240]:
57600000/3_600_000

16.0

In [249]:
def identify_length(string, integer, rt=False):
    if rt:
        TIMEFRAMES_VALUES = {'m': 1, 'h': 60, 'd': 60*24, 'w': 60*24*7}
    else:
        TIMEFRAMES_VALUES = {'d': 1, 'w': 5, 'm': 30, 'y': 252, 'q': 91}
    assert string in TIMEFRAMES_VALUES.keys(
    ), f'Available timeframes are {TIMEFRAMES_VALUES.keys()}, recieved "{string}"'
    return integer * TIMEFRAMES_VALUES[string]
identify_length(*extract_numeric_value('1h'), rt=True)*60000


def convert_time_to_miliseconds(time):
    time_obj = pd.to_datetime(time)
    hour = time_obj.hour * 3_600_000
    minute = time_obj.minute * 60_000
    secs = time_obj.second * 1_000
    mili = time_obj.microsecond
    return hour + minute + secs + mili

convert_time_to_miliseconds('9:30')

34200000

In [139]:
order_candidates['long'][0]

right,expiration,DTE,strike,relative_moneyness,moneyness_spread,dte_spread,ticker,Spot,moneyness,TGT_DTE,q,r,right,option_id,liquidity_check
build_date,,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,190.0,1.096808,0.009372,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000190P,False
2024-03-12,2025-03-21,373,185.0,1.067944,0.004616,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000185P,False
2024-03-12,2025-03-21,373,180.0,1.039081,0.001527,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000180P,True
2024-03-12,2025-03-21,373,175.0,1.010218,0.000104,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000175P,False
2024-03-12,2025-03-21,373,170.0,0.981354,0.000348,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000170P,True
2024-03-12,2025-03-21,373,165.0,0.952491,0.002257,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000165P,True
2024-03-12,2025-03-21,373,160.0,0.923628,0.005833,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000160P,False


In [137]:
oi = liquidity_check('AAPL20250321000185P', date)
oi

False

In [75]:
sample_id = deepcopy(get_option_specifics_from_key('AAPL20250321000165P'))
new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
transfer_dict = {}
for k, v in sample_id.items():
    print(k)
    if k in new_dict_keys:
        if k == 'strike':
            transfer_dict[new_dict_keys[k]] = float(sample_id[k])
        else:
            transfer_dict[new_dict_keys[k]] = (sample_id[k])

sample_id

ticker
put_call
exp_date
strike


{'ticker': 'AAPL', 'put_call': 'P', 'exp_date': '2025-03-21', 'strike': 165.0}

In [76]:
close_data_sample = retrieve_eod_ohlc(**transfer_dict, start_date=start, end_date=date)
close_mask_series = close_data_sample.Close != 0
close_mask_series.sum()/len(close_mask_series)
# close_data_sample.Close

0.8

In [210]:
available_close_check('AAPL20250321000160P', date)

0.7333333333333333

In [81]:
close_data_sample['Midpoint'][date]

9.85

In [98]:
sample_id = deepcopy(get_option_specifics_from_key('AAPL20250321000160P'))
sample_id

{'ticker': 'AAPL', 'put_call': 'P', 'exp_date': '2025-03-21', 'strike': 160}

In [ ]:
retrieve_openInterest

Signature:
retrieve_openInterest(
    symbol,
    end_date: str,
    exp: str,
    right: str,
    start_date: str,
    strike: float,
    print_url=False,
    proxy='http://18.232.166.224:5500/thetadata',
)
Docstring: Interval size in miliseconds. 1 minute is 6000
File:      ~/cloned_repos/FinanceDatabase/dbase/DataAPI/ThetaData.py
Type:      function

In [37]:

chain_details?

Signature:
chain_details(
    date,
    ticker,
    tgt_dte,
    tgt_moneyness,
    right='P',
    moneyness_width=0.15,
)
Docstring: <no docstring>
File:      /var/folders/j0/80hkbygd4lb27h9mw76gqzpw0000gn/T/ipykernel_16186/3700718713.py
Type:      function

Steps to producing an order:

- S1: RM recieves order settings from PM
- S2: RM produces a dataframe of potential options based on settings (if two legs produce two dataframes)
- S3: RM assesses if option passes all checks
    - C1: Minimum Available close
    - C2: Liquidity (Open Interest)
    - C2.5: (for Spreads only) Ensure both legs are not the same
    - Optional, to extend:
    - C3: Bid-Ask Spread
    
- S4: Return picked order to portfolio manager, which places the order. 
- Example:
    {'long': [optionid or {'strike', 'exp'}], 'short' : []}

In [65]:
generate_option_tick('AAPL','P' ,'2025-03-21', 150.0)
# save_option_keys(generate_option_tick('AAPL','P' ,'2025-03-21', 150.0), {'ticker':'AAPL', 'put_call':'P', 'exp_date':'2025-03-21', 'strike':150.0})

'AAPL20250321000150P'

In [256]:
opt = Option(**option_keys['AAPL20250321000150P'], run_chain = True)

NameError: name 'option_keys' is not defined

In [39]:
import_option_keys()
option_keys

{}